### Hyun-Joon Yang
### yanghyun@usc.edu
### QBIO 401
### Assignment 6

On Blackboard you can find a file named: “abbgen1k.csv”. This file is a subset of the 1,000 Genomes Project for chromosome 22. The format is the same as we discussed in class: rows are SNPs, columns 1 to 9 (R starts with 1) are details about the SNPs, columns 10 to 90 are unrelated individuals from Europe, and columns 91 to 179 are unrelated individuals from Africa.

Compute the average pairwise diversity within the European samples. Also, compute this within the African samples, and between the European and African samples. Due to the out-of-Africa hypothesis, we expect that the average pairwise diversity (average number of different SNPs between two individuals) within Europe will be less than that within African. 

#### 1. Write an R function that inputs a vector representing the genotype of an individual and a logical indicating “left” or “right” haplotype. This function returns either the “left” or “right” haplotype as a vector. [2pt].

In [2]:
# genotype: vector of strings for genotype
# getLeft: True if left, False if right haplotype
# return: vector of haplotype
getHaplotype <- function(genotype, getLeft) {
    strlist <- strsplit(genotype, '|', fixed=T)
    # function for lapply to extract left or right
    getLeftOrRight <- function(vec, getLeft) {
        if(getLeft == T) {
            return(vec[1])
        } else {
            return(vec[2])
        }
    }
    # turn list into vector then return
    result <- unlist(lapply(strlist, getLeftOrRight, getLeft))
    return(result)
}

TEST

In [3]:
data <- read.csv('abbgen1k.csv', stringsAsFactors = F)
head(data)

CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA06984,...,NA19143,NA19144,NA19146,NA19147,NA19113,NA19114,NA19256,NA19257,NA19117,NA19118
22,16050115,rs587755077,G,A,100,PASS,AC=32;AF=0.00638978;AN=5008;NS=2504;DP=11468;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0234;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050213,rs587654921,C,T,100,PASS,AC=38;AF=0.00758786;AN=5008;NS=2504;DP=15092;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0272;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,1|0,1|0,0|0,0|0,0|0
22,16050783,rs587743568,A,G,100,PASS,AC=39;AF=0.00778754;AN=5008;NS=2504;DP=24717;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0287;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050840,rs587616822,C,G,100,PASS,AC=26;AF=0.00519169;AN=5008;NS=2504;DP=23832;EAS_AF=0;AMR_AF=0.0058;AFR_AF=0;EUR_AF=0.0209;SAS_AF=0.001;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050958,rs587636807,A,T,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=22326;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050984,rs188945759,C,G,100,PASS,AC=20;AF=0.00399361;AN=5008;NS=2504;DP=22013;EAS_AF=0;AMR_AF=0;AFR_AF=0.0151;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [4]:
# test on simple vector
test <- c('0|1', '1|0', '0|0', '1|1')
haplotype <- getHaplotype(test, T)
print(all(haplotype == c('0', '1', '0', '1')))
haplotype <- getHaplotype(test, F)
print(all(haplotype == c('1', '0', '0', '1')))

[1] TRUE
[1] TRUE


In [5]:
head(data$NA19256)
haplotype <- getHaplotype(data$NA19256, T)
print(all(haplotype[1:6] == c('0', '1', '0', '0', '0', '0')))

[1] "0|0" "1|0" "0|0" "0|0" "0|0" "0|0"

[1] TRUE


#### 2. Write an R function that computes the average pairwise diversity for n randomly chosen pairs of haplotypes (use the built-in R function ”sample”). These pairs of haplotypes can both be chosen from one sample, or they can be chosen from two different samples. The inputs to this function are a dataframe with the format in “abbgen1k.csv”, a vector of column numbers for individuals in the first sample, a vector of column numbers for individuals in the second sample, and a number n [3pt].

In [2]:
# data: dataframe of the data in VCF format
# cols1: vector of column numbers for individuals in the first sample
# cols2: vector of column numbers for individuals in the second sample
# n: number of randomly chosen pairs
computeDiversity <- function(data, cols1, cols2, n) {
    # combine cols for random sampling
    cols <- c(cols1, cols2)
    count <- 0
    # calculate diversity n times
    for(i in 1:n) {
        # get haplotype randomly
        chosen <- sample(cols, 2)
        isLeft <- sample(0:1, 2)
        hap1 <- getHaplotype(data[,chosen[1]], isLeft[1])
        hap2 <- getHaplotype(data[,chosen[2]], isLeft[2])
        count <- count + sum(hap1 != hap2)
    }
    return(count/n)
}

#### 3. Run Function #3 for three comparisons including within the European samples (with the input the dataframe columns 10:90 and 10:90), the African samples (with the input the dataframe columns 91:179 and 91:179), and between the European and African samples (with the input dataframe columns 10:90 and 91:179). Set the parameter n equal to 1000. Tell us what you find from the results and explain whether the results can support the out-of-Africa hypothesis [3pt].

In [3]:
data <- read.csv('abbgen1k.csv', stringsAsFactors = F)
head(data)

n <- 1000
euro_euro <- computeDiversity(data, 10:90, 10:90, n)
afro_afro <- computeDiversity(data, 91:179, 91:179, n)
euro_afro <- computeDiversity(data, 10:90, 91:179, n)
afro_euro <- computeDiversity(data, 91:179, 10:90, n)

print(paste(euro_euro, afro_afro, euro_afro, afro_euro))

CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA06984,...,NA19143,NA19144,NA19146,NA19147,NA19113,NA19114,NA19256,NA19257,NA19117,NA19118
22,16050115,rs587755077,G,A,100,PASS,AC=32;AF=0.00638978;AN=5008;NS=2504;DP=11468;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0234;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050213,rs587654921,C,T,100,PASS,AC=38;AF=0.00758786;AN=5008;NS=2504;DP=15092;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0272;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,1|0,1|0,0|0,0|0,0|0
22,16050783,rs587743568,A,G,100,PASS,AC=39;AF=0.00778754;AN=5008;NS=2504;DP=24717;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0.0287;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050840,rs587616822,C,G,100,PASS,AC=26;AF=0.00519169;AN=5008;NS=2504;DP=23832;EAS_AF=0;AMR_AF=0.0058;AFR_AF=0;EUR_AF=0.0209;SAS_AF=0.001;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050958,rs587636807,A,T,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=22326;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.001;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
22,16050984,rs188945759,C,G,100,PASS,AC=20;AF=0.00399361;AN=5008;NS=2504;DP=22013;EAS_AF=0;AMR_AF=0;AFR_AF=0.0151;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


[1] "1483.726 1808.213 1747.203 1773.922"


It seems there is more diversity within Africa than within Europe and between Europe and Africa which supports the out-of-Africa hypothesis.